**Load packages**

In [1]:
#OS
import os
# Pipelines
from sklearn.pipeline import Pipeline 
# Pre-processing
from sklearn.preprocessing import StandardScaler # (x-u)/s
# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# Models Selection
from sklearn.model_selection import GridSearchCV
# Models Metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score


In [1]:
#OS
import os

#Manipulación de Datos
import pandas as pd
import numpy as np
# Pre-processing
from sklearn.preprocessing import StandardScaler # (x-u)/s
# Models Clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

**Feature Selection for clustering models**

In [1]:
def FeatureSelectionForClustering(df,maxVars=15,kmInit="random",kmNclusters=6,kmNinit=10,kmMax_iter=300,kmRandomState=42):
    #
    featureList=df.columns.tolist()
    cols_iter=1
    selectedFeatures = []
    
    dfFeatures = pd.DataFrame({'Order': pd.Series(dtype='int'),
                                 'Feature': pd.Series(dtype='str'),
                                 'Silhouette': pd.Series(dtype='float')})
    
    # Loop to get list of the best features for Clustering
    while(len(selectedFeatures)<maxVars):
        # DF to append features to test
        costFeatures = pd.DataFrame({'Feature': pd.Series(dtype='str'),
                                     'Silhouette': pd.Series(dtype='float')})
        # Counter to show remaining quantity of features
        cols_iter=1
        for i in featureList:
            colsTesting=selectedFeatures.copy()  # Colect the best list of feature
            colsTesting.append(i) # Append
            # Subset
            dataSet=df[colsTesting] #subset with testing
            # Creating K-means Object and Training model
            kmeans = KMeans(init="random",
                            n_clusters=6,
                            n_init=10,
                            max_iter=300,
                            random_state=42)
            kmeans.fit(dataSet)
            # Silhouette score
            score = silhouette_score(dataSet, kmeans.labels_)
            # Temp Silhouette Dataframe
            temp_df = pd.DataFrame({
                'Feature': [i],
                'Silhouette': [score]})
            costFeatures=costFeatures.append(temp_df) # Collecting features
            cols_iter=cols_iter+1
        # End For
        bestSilhouette=costFeatures
        bestSilhouette=pd.merge(bestSilhouette, describeCommons(df), how="inner", on=["Feature"])
        # Preparing to select the best Feature
        bestSilhouette.sort_values(["Silhouette", "%MostCommon"],
                                   axis = 0, ascending = [False, True],
                                   inplace = True,
                                   ignore_index=True,
                                   na_position = "first")
        print('\n')
        #print(bestSilhouette)
        # Best feature
        bestSilhouetteFeature=bestSilhouette.iloc[0,0]      # feature
        bestSilhouetteMetric =str(bestSilhouette.iloc[0,1]) # silhouette
        # Append Best Feature in a list
        selectedFeatures.append(bestSilhouetteFeature)
        # Remove best feature / update list
        featureList.remove(bestSilhouetteFeature)
        # Append Features in DF
        tempFeatures = pd.DataFrame({
            'Order': [len(selectedFeatures)],
            'Feature': [bestSilhouetteFeature],
            'Silhouette': [bestSilhouetteMetric]})
        dfFeatures=dfFeatures.append(tempFeatures) # Collecting features    
        # Print the best feature / silhouette
        print('The number of features selected until now is : '+str(len(selectedFeatures)))
        print('The feature(s) with the best silhouette is **{}** with {}'.format(bestSilhouetteFeature,bestSilhouetteMetric))
    
    return dfFeatures,selectedFeatures
print('Loading Feature Selection for clustering models...')

Loading Feature Selection for clustering models...
